# Probabilistic sensitivity analysis: most probable distributed parameters
In probabilistic sensitivity analyses distributions of baseline parameters are resampled multiple times from a distribution. Subsequently, the Markov model is implemented for the defined simulation period and cumulative outcomes for the cohort are computed (QALYs: Quality-Adjusted Life Years, costs in €, NMB: Net Monetary Benefit in €). This is often referred to as the term Markov Chain Monte Carlo simulations. The computations below implement draws of model parameters from the most probable distribution. The data used refers to the study (appendix D includes all parameters used): <br>

"Cost and health effects of case management compared to outpatient clinic follow-up in a Dutch heart failure cohort" <br> by H. van Voorst and A.E.R. Arnold.<br>
DOI: 10.1002/ehf2.12692 <br>

This notebook is the last in an series of three:
1. Baseline simulations and one-way deterministic sensitivity analyses.
2. Probabilistic sensitivity analysis: uniform distributed parameters
3. Probabilistic sensitivity analysis: most probable distributed parameters

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
np.random.seed(24)
import math
import time

## Defining the transition probabilities

In [2]:
# draw RR from log-normal distribution
def RR_draw(RR, RR_p025, RR_p975):
    """
    Draws Relative Risk (RR) from a log-normal
    distribution according to the description
    in Higgs et al.
    
    Input: mean RR and lower-upper bound of 95% CI
    
    Output: RR draw from log normal distribution
    """
    
    #log transform than draw from normal dist
    mean = math.log(RR)
    sd = (math.log(RR_p975)-math.log(RR_p025))/(2*1.96)
    RR_draw = math.exp(np.random.normal(mean, sd))
    return RR_draw


# compute monthly number of events
def monthly_n_events(total_pt,event_contr, 
                     months, RR,rr_months, probs=False):
    
    """
    Computes the probability or number of events
    per months for for control and intervention arm.
    
    Input:
    Total amount of patients in control arm (total_pt)
    Number of control arm patients with event (event_contr)
    Time span in months for event measurement (months)
    Relative Risk of event in interventnion arm (RR)
    Time span in months for RR measurement (rr_months)
    Possibility to return probabilities or number
    of events (probs=True returns probabilities)
    
    p_control: Probability of event in control arm
    RR: Relative Risk of event in intervention 
    arm relative to control arm
    rr_months: months used to compute RR
    pc_months: months over which p_control is measured
    
    Output: probability intervention arm in pc_months
    """
    p_eventfree = 1-(event_contr/total_pt)
    adj_eventfree = p_eventfree**(rr_months/months)
    p_intervention_rrm = RR*(1-adj_eventfree)
    
    monthly_p_control = 1-(1-(event_contr/total_pt))**(1/months)
    monthly_p_intervention = 1-(1-p_intervention_rrm)**(1/rr_months)
    
    #return monthly events
    control_out = monthly_p_control*total_pt
    intervention_out = monthly_p_intervention*total_pt
    
    if probs==True:
        control_out = monthly_p_control
        intervention_out = monthly_p_intervention
    
    return control_out, intervention_out


# Draw intervention and control probability from beta distribution
def probdraw_ci_RR(control_events, tot, months, RR, rr_months, probs=True): 
    """
    Draws probability of events based on number
    of events for control and intervention arm.
    Implicitly RR is used to compute intervention arm
    probability
    
    Input: 
    Number of control arm events (control_events)
    Total amount of patients in control arm (tot)
    Time span in months for event measurement (months)
    Relative Risk of event in interventnion arm (RR)
    Time span in months for RR measurement (rr_months)
    Possibility to return probabilities or number
    of events (probs=True returns probabilities)
    
    Output: Control and intervention arm pobabilities/n events
    in amount of months used for control arm measurements
    """

    prob_control = np.random.beta(control_events, tot-control_events)
    #comput new drawn event rate to compute intervention event rate
    control_events = prob_control*tot 
    # now draw a intervention probability adjusted for the month rate; 
    prob_contr, prob_int  = monthly_n_events(tot,control_events, months, 
                                             RR, rr_months, probs=probs)
    
    return prob_contr, prob_int

def monthly_event_sep(tot, event,months):
    """
    Computes monthly amount of events,
    used for dirichelet draws with 
    monthly rates
    
    Input: total number of patients, 
    number of patients with even,
    measured month span of events
    
    Output: number of monthly events
    """
    
    p = event/tot
    p_monthly = 1-(1-p)**(1/months)
    n_monthly = p_monthly*tot
    return n_monthly


In [3]:
# transition probabilities: ai = a intervention, ac = a control
def draw_probabilities():
    """
    Draws all probabilities in the model
    for both control and intervention arm

    """
    
    #Transform RR to log scale and fit a 
    # normal distribution (according to Briggs et al.)
    RR_readmission,RR_mortality=0.64,0.78
    RR_readmin, RR_readmax, RR_mortmin, RR_mortmax = 0.53,0.78,0.68,0.90
    RR_months = 12
    
    # draw the values of RR from lognormal dist
    RR_readmission = RR_draw(RR_readmission, RR_readmin, RR_readmax)
    RR_mortality = RR_draw(RR_mortality, RR_mortmin, RR_mortmax)
    
    #B = N12 -> N34 (NYHA decay from NYHA 1/2 to 
    #NYHA 3/4; net effect assumed zero)
    bc_draw = 0
    bi_draw = 0
    
    #A = N12 -> N12 (no change from NYHA 1/2)
    #C = N12 -> H (Hospital readmission from NYHA 1/2)
    #D = N12 -> D (Mortality from NYHA 1/2)
    # a,c,d were drawn joint from a dirichelet distribution based on the counts
    # intervention event amounts were computed from the control events and the RR
    tot_acd = 948
    # compute monthly event rate
    c_event = monthly_event_sep(tot_acd, 185,12)
    d_event = monthly_event_sep(tot_acd, 217,12)
    a_event = tot_acd - c_event - d_event
    ac_draw, cc_draw, dc_draw = \
    np.squeeze(np.random.dirichlet((a_event,c_event,d_event), 1))
    
    # for intervention arm first compute expected events with RR
    __,ci_event = probdraw_ci_RR(185, tot_acd, 12, RR_readmission, 12, probs=False)
    __,di_event = probdraw_ci_RR(217, tot_acd, 12, RR_mortality, 12, probs=False)
    ai_event = tot_acd - ci_event - di_event
    ai_draw, ci_draw, di_draw = np.squeeze(np.random.dirichlet((ai_event,ci_event,di_event), 1))

    #E = N34 -> N12 (No recovery from NYHA 3/4 to 
    #NYHA 1/2 was assumed 0)
    ec = 0
    ei = 0
    
    #F = N34 -> N34 (resultant; No change from NYHA 3/4)
    #G = N34 -> H (Hospital readmission rate from NYHA 3/4)
    #H = N34 -> D (Mortality rate from NYHA 3/4)
    tot_fgh = 78
    # compute monthly event counts 
    g_event = tot_fgh*0.0227 #see table 1 in article (probability from literature)
    h_event = monthly_event_sep(tot_fgh, 31,12)
    f_event = tot_fgh - g_event - h_event
    # draw monthly control arm probabilities from dirichelet distribution
    fc_draw, gc_draw, hc_draw = \
    np.squeeze(np.random.dirichlet((f_event,g_event,h_event), 1))
    # Compute the intervention arm events
    __,gi_event = probdraw_ci_RR(g_event, tot_fgh, 1, RR_readmission, 12, probs=False)
    __,hi_event = probdraw_ci_RR(31, tot_fgh, 12, RR_mortality, 12, probs=False)
    fi_event = tot_fgh - gi_event - hi_event
    # draw monthly intervention arm probabilities from dirichelet distribution
    fi_draw, gi_draw, hi_draw = np.squeeze(np.random.dirichlet((fi_event,gi_event,hi_event), 1))

    
    #I = H -> N12 (resultant; discharge to NYHA 1/2)
    #J = H -> N34 (Discharge to NYHA 3/4)
    #L = H -> D (In hospital mortality)
    # discharge allocation does not vary between intervention and control group
    # I, J, and L were drawn joint from a dirichelet distribution
    i_event,j_event,l_event = 948,78,88
    ic_draw, jc_draw, lc_draw = np.squeeze(np.random.dirichlet((i_event,j_event,l_event), 1))
    ii_draw, ji_draw, li_draw = ic_draw, jc_draw, lc_draw
    
    #K = H -> H (Hospital admission were 
    # not longer than 1 month; defined 0)
    kc = 0
    ki = 0
    
    #M,N,O define as zero, P defined as 1 (dead = dead)
    
    return ac_draw, bc_draw, cc_draw, dc_draw, \
    ec, fc_draw, gc_draw, hc_draw, ic_draw, jc_draw, kc, lc_draw,\
    ai_draw, bi_draw, ci_draw, di_draw, \
    ei, fi_draw, gi_draw, hi_draw, ii_draw, ji_draw, ki, li_draw 

    


In [4]:
# inflation adjustment factor computation
def infl_adjustment(months, yearly_CPI=1.029): 
    """
    Compute a inflation adjustment factor for the amount
    of months (months) that have passed since the 
    start year (reference year; 2020). Use a predefined
    inflation factor (yearly_CPI).
    
    Output: The inflation adjustment factor.
    """
    CPI_adj_factor = (yearly_CPI)**(months/12)
    return CPI_adj_factor

def draw_costs(ci, CPI, refyear):
    """
     Draws costs from gamma distribution with
     pre specified distribution parameters

    - ic: Either the 'Intervention' or 'Control' arm 
        as follow-up costs can differ.
    - CPI: Consumer price index adjustment factor, 
        used to compute the current
        costs indexed from the year in which costs were computed.
        In the study either 2014 or 2016 were 
        used for different costs.
    - refyear: The reference case year in which the simulations start,
        in the study 2020 was used.

    Output: Cost per month for each of the 4 Markov States
    """
    
    if ci== 'Intervention':
        fu_cost = np.random.gamma(shape = 1.8, scale = 20)
        fu_cost = round(fu_cost*(CPI**(refyear-2014)),2)
        
    elif ci=='Control':
        fu_cost = np.random.gamma(shape = 2.0, scale = 18)
        fu_cost = round(fu_cost*(CPI**(refyear-2016)),2)
        
    C_N12 = fu_cost
    C_N34 = fu_cost
    
    C_H_shape = 3.8
    C_H_scale = 1/0.001
    C_H = np.random.gamma(shape = C_H_shape, scale = C_H_scale)
    C_H = round(C_H*(CPI**(refyear-2016)),2)
    
    C_D = 0
    
    return C_N12, C_N34, C_H, C_D

def QALYmonth(n_tot, QALY_y):
    """
     Draws QALYs per month from a 
     beta distribution with pre-
     specified distribution parameters

    Input:
    Total amount of measurements from literature (n_tot)
    Yearly QALYs (QALY_y)

    Output: Monthly QALY drawn from beta distribution
    """
    n1 = n_tot*QALY_y
    m_QALY = np.random.beta(n1, n_tot-n1)/12
    return m_QALY
    
def draw_QALYs():
    
    """
    Define monthly QALYs for the 4 Markov 
    states used in this model drawn from beta
    distributions.
    
    Output: QALYs per month for each of the 4 Markov states
     """
    
    QN12_tot = 2518
    QALY_N12_y = 0.76
    Q_N12 = QALYmonth(QN12_tot, QALY_N12_y)
    
    QN34_tot = 2795
    QALY_N34_y = 0.54
    Q_N34 = QALYmonth(QN34_tot, QALY_N34_y)

    Q_H = QALYmonth(QN34_tot, QALY_N34_y)
    
    Q_D = 0
    
    return Q_N12, Q_N34, Q_H, Q_D



In [5]:
# same function as in baseline model
def simulate_month(df, # pd DataFrame where all results are stored
                   r, # A name to add to each row of new results in df
                   month, # the period (month) since start of simulation
                   patient_dist, # Markov state distribution before new period
                   transition_mat, # Matrix with transition probabilities
                   Q_mat, # Matrix with QALYs per Markov state
                   C_mat, # Matrix with Costs per Markov state
                   discount_rate_C, # Discounting % for costs
                   CPI, # Inflation rate
                   discount_rate_Q): # Discounting % for QALYs
    """
    Simulates a single month given input parameters
    
    Output: df with results (df) and new Markov state
    distribution of patients
    """
    
    # Compute inflation adjustment factor (CPI_adj)
    # for the amount of months that have passed since 
    # the begin of simulations
    CPI_adj = infl_adjustment(month, CPI)
    
    # compute the patient Markov state distribution after 1 period (month)
    new_patient_dist = np.matmul(patient_dist,transition_mat)
    
    # Use the patient Markov state distribution 
    #to compute costs and QALYs for the specified period (month)
    QALYs = new_patient_dist*Q_mat
    Costs = new_patient_dist*C_mat
    T_Q = QALYs.sum()
    T_C = Costs.sum()
    # Compute discounted costs and QALYs
    disc_factor_C = discount_rate_C**(month/12)
    disc_factor_Q = discount_rate_Q**(month/12)
    disc_Q = T_Q/(disc_factor_Q )
    disc_C = round((T_C*CPI_adj)/(disc_factor_C), 2)
    
    # put everything in a new row in the dataframe
    nr = [r, month, *new_patient_dist, 
          *QALYs, T_Q, disc_Q, 
          *Costs, T_C, disc_C]
    df.loc[len(df)]=nr
    
    return df, new_patient_dist


## Perform the simulation


In [6]:
# note: Q and C are now drawn once while in previous versions those differed?
def simulate_prob_dist(repeats):
    """
    Simulates results for control and 
    intervention arm based on pre-defined 
    amount of repeats.
    
    Output: df with results (df) and new Markov state
    distribution of patients
    """
    
    t1 = time.time()

    sim_months = 60

    discount_rate_C = 1.04
    discount_rate_Q = 1.015
    CPI = 1.029
    refyear=2020

    cohort_size = 1e5
    cols = ['Repeat', 'Month', 'NYHA_12', 'NYHA_34', 'Hospital', 'Dead', 
                'Q_N12','Q_N34','Q_H', 'Q_D', 'QALY_tot', 'QALY_disc',
                'C_N12', 'C_N34','C_H', 'C_D', 'Cost_tot', 'Cost_disc']

    control_result_df = pd.DataFrame(columns = cols)
    intervention_result_df = pd.DataFrame(columns = cols)
    t2 = time.time()
    for r in range(0,repeats):
        # for each repeat restart the cohort distribution
        # and draw all new parameters
        cohort_dist_control = np.array([0,0,cohort_size,0])
        cohort_dist_intervention = np.array([0,0,cohort_size,0])
        
        # input probabilities for events in transition matrices
        ac, bc, cc, dc, ec, fc, gc, hc, ic, jc, kc, lc,\
        ai, bi, ci, di, ei, fi, gi, hi, ii, ji, ki, li = draw_probabilities()
    
        # QALY anc Cost matrices
        QALY_N12c, QALY_N34c, QALY_Hc, QALY_Dc = draw_QALYs()
        Q_matc = np.array([QALY_N12c,QALY_N34c,QALY_Hc,QALY_Dc])
        Q_mati = Q_matc 
        C_matc = draw_costs('Control', CPI, refyear)
        C_mati = draw_costs('Intervention', CPI, refyear)
        
        # transition matrices for control
        # and intervention arm
        trmat_control = \
        np.array([[ac, bc, cc, dc], 
                  [ec, fc, gc, hc], 
                  [ic, jc, kc, lc],
                  [0,0,0,1]], dtype = 'float64')
        trmat_intervention = \
        np.array([[ai, bi, ci, di], 
                  [ei, fi, gi, hi], 
                  [ii, ji, ki, li],
                  [0,0,0,1]], dtype = 'float64')

        for month in range(1,sim_months+1):
            # compute monthly results
            control_result_df, cohort_dist_control = \
            simulate_month(control_result_df,r,month,
                           cohort_dist_control,trmat_control, 
                           Q_matc, C_matc, 
                           discount_rate_C, CPI, discount_rate_Q)
            
            intervention_result_df, cohort_dist_intervention = \
            simulate_month(intervention_result_df,r,month,
                           cohort_dist_intervention,trmat_intervention, 
                           Q_mati, C_mati, 
                           discount_rate_C, CPI, discount_rate_Q)
            
        t3a=time.time()
        if (r%200==0) & (r!=0):
            print('Finished repeat number:', r, 'running time', round((t3a-t2),2), 'seconds')
        #if r ==20:
         #   break
    t3 = time.time()   
    print('Total simulation time:', round((t3-t1),2), 'seconds')    
    return control_result_df, intervention_result_df
                

In [ ]:
def delta_outcome(dfc, dfi, WTP):
    """
    Computes differences between control
    and intervention arm for each repeat 
    (row in dfc and dfi)
    
    Input:
    Control arm df (dfc)
    Intervention arm df (dfi)
    Willingness to pay per QALY (WTP)
    
    Output: df with all differences and computed
    per state outcomes 
    """
    
    reps = dfi.Repeat.unique()
    
    cols =['Repeat','control_costs', 'intervention_costs', 'dcosts', 
           'control_QALYs', 'intervention_QALYs', 'dQALYs', 'NMB',
           'survi','survc','N12i','N12c','N34i','N34c', 'Hi', 'Hc']
    df_out = pd.DataFrame(columns = cols)
    
    for r in reps:
        intervention = dfi[dfi.Repeat == r]
        control = dfc[dfc.Repeat == r]
        
        costi = intervention.Cost_disc.sum()/1e5
        costc = control.Cost_disc.sum()/1e5
        dcost = costi - costc
                
        QALYi = intervention.QALY_disc.sum()/1e5
        QALYc = control.QALY_disc.sum()/1e5
        dQALY = QALYi - QALYc
        NMB = WTP*dQALY-1*dcost
        
        survi = ((intervention.Dead*-1+1e5)).sum()/1e5
        survc = ((control.Dead*-1+1e5)).sum()/1e5
        
        N12i = intervention.NYHA_12.sum()/1e5
        N12c = control.NYHA_12.sum()/1e5
        
        N34i = intervention.NYHA_34.sum()/1e5
        N34c = control.NYHA_34.sum()/1e5
        
        Hi = intervention.Hospital.sum()/1e5
        Hc = control.Hospital.sum()/1e5
        
        df_out.loc[len(df_out)] = [r, costc, costi, dcost, QALYc, QALYi, dQALY, NMB,
                                  survi,survc,N12i,N12c,N34i,N34c, Hi, Hc]
        
    return df_out

# implement all:
dfc,dfi = simulate_prob_dist(1000)
df_d = delta_outcome(dfc, dfi, 50000)
print(df_d.dcosts.values.max(), df_d.dcosts.values.min(), df_d.dcosts.values.mean(), df_d.dcosts.values.std())
df_d.head()